# Imports

In [71]:
# %pip install --upgrade z3-solver

In [111]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_wine
from sklearn.datasets import load_digits
from pmlb import fetch_data
from z3 import *

In [112]:
set_option(rational_to_decimal=True)

# Z3 Functions

(I ^ T -> D)

## function I (instance expression)

In [74]:
def instance_expression(instance):
    formula = [Real(f'x{i}') == value for i, value in enumerate(instance)]
    return formula

## funcion T (tree leafs and constraints expression)

T = T_model ^ T_constraints

In [75]:
def feature_constraints_expression(X):
    constraints = []

    for i in range(X.shape[1]):
        feature_values = X[:, i]
        min_val, max_val = feature_values.min(), feature_values.max()

        x = Real(f'x{i}')
        min = RealVal(min_val)
        max = RealVal(max_val)

        constraint = And(min <= x, x <= max)
        constraints.append(constraint)

    return And(*constraints)

In [76]:
def tree_paths_expression(tree, tree_index, class_index):
    tree_ = tree.tree_
    feature = tree_.feature
    threshold = tree_.threshold
    value = tree_.value

    paths = []
    o = Real(f'o_{tree_index}_{class_index}')

    def traverse(node, path_conditions):

        if feature[node] == -2:
            leaf_value = value[node][0][0]
            path_formula = And(path_conditions)
            implication = Implies(path_formula, o == leaf_value)
            paths.append(implication)
        else:

            x = Real(f'x{feature[node]}')
            left_condition = x <= threshold[node]
            right_condition = x > threshold[node]
            traverse(tree_.children_left[node],
                     path_conditions + [left_condition])
            traverse(tree_.children_right[node],
                     path_conditions + [right_condition])

    traverse(0, [])
    return And(*paths)

In [77]:
def model_trees_expression(model):
    formulas = []
    for i, estimators in enumerate(model.estimators_):
        for class_index, estimator in enumerate(estimators):
            formula = tree_paths_expression(estimator, i, class_index)
            formulas.append(formula)
    return And(*formulas)

## function D (decision function result expression)

In [181]:
def decision_function_expression(model, x):
    learning_rate = model.learning_rate
    estimators = model.estimators_
    n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_
    
    decision = model.decision_function(x)
    predicted_class = model.predict(x)[0]

    estimator_results = []
    for estimator in estimators:
        class_predictions = [tree.predict(x) for tree in estimator]
        estimator_results.append(class_predictions)

    estimator_sum = np.sum(estimator_results, axis=0) * learning_rate
    init_value = decision - estimator_sum.T

    equation_list = []
    for class_number in range(n_classes):
        estimator_list = []
        for estimator_number in range(len(estimators)):
            o = Real(f"o_{estimator_number}_{class_number}")
            estimator_list.append(o)
        equation_o = Sum(estimator_list) * learning_rate + \
            init_value[0][class_number]
        equation_list.append(equation_o)

    if n_classes <= 2:
        if predicted_class == 0:
            final_equation = equation_list[0] < 0
        else:
            final_equation = equation_list[0] > 0
    else:
        compare_equation = []
        for class_number in range(n_classes):
            if predicted_class != class_number:
                compare_equation.append(
                    equation_list[predicted_class] > equation_list[class_number]
                )
        final_equation = compare_equation

    return And(final_equation)

# Explaination Functions

In [79]:
def is_proved(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        return True
    else:
        return False

In [80]:
def explain(I, T, D, model, reorder):
    X = I.copy()

    importances = model.feature_importances_
    if reorder == 'asc':
        sorted_feature_indices = np.argsort(importances)
        X = [X[i] for i in sorted_feature_indices]
    elif reorder == 'desc':
        sorted_feature_indices = np.argsort(np.flip(importances))
        X = [X[i] for i in sorted_feature_indices]

    for feature in X.copy():
        X.remove(feature)

        if is_proved(Implies(And(And(X), T), D)):
            continue
            # print('proved')
        else:
            # print('not proved')
            X.append(feature)

    return X

In [81]:
class Explainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data
        self.T_constraints = feature_constraints_expression(self.data)
        self.T_model = model_trees_expression(self.model)
        self.T = And(self.T_model, self.T_constraints)

    def explain(self, instance, reorder='asc'):
        D = decision_function_expression(self.model, [instance])
        I = instance_expression(instance)
        return explain(I, self.T, D, self.model, reorder)

In [82]:
def explain_instance(model, data, instance):
    I = instance_expression(instance)
    T_constraints = feature_constraints_expression(data)
    T_model = model_trees_expression(model)
    T = And(T_model, T_constraints)
    D = decision_function_expression(model, [instance])

    return explain(I, T, D)

In [83]:
# gb = GradientBoostingClassifier(n_estimators = 10)
# breast_cancer = load_breast_cancer()
# X_train, X_test, y_train, y_test = train_test_split(breast_cancer.data, breast_cancer.target, test_size=0.1, random_state=101)
# gb.fit(X_train, y_train)

In [84]:
# test = explain_instance(gb, breast_cancer.data, X_test[0])
# print(test)

## outro

In [85]:
def multiclass_initial_prediction(y, n_classes):
    class_counts = np.bincount(y, minlength=n_classes)
    class_probs = class_counts / len(y)

    log_odds = np.log(class_probs + 1e-15)
    return log_odds - log_odds.mean()

In [86]:
def binary_initial_prediction(y):
    pi = np.mean(y)

    initial_log_odds = np.log(pi / (1 - pi))

    return initial_log_odds

In [87]:
def compare_decision_function(model, X, initial_prediction=None):
    learning_rate = model.learning_rate
    estimators = model.estimators_
    estimator_results = []

    if initial_prediction is None and model.init_ != 'zero':
        print("Error - Missing initial_prediction")
        return None

    for estimator in estimators:
        class_predictions = [tree.predict(X) for tree in estimator]
        estimator_results.append(np.array(class_predictions).T)

    final_predictions = np.sum(estimator_results, axis=0) * learning_rate

    if model.init_ != 'zero':
        final_predictions += initial_prediction

    if np.isscalar(initial_prediction) and initial_prediction != None:
        final_predictions = final_predictions.flatten()

    if not np.allclose(final_predictions, model.decision_function(X)):
        print("Error - Deicison Function does not match")

    return final_predictions

In [88]:
def print_init_decision_function(model, X):
    X = [X[1]]

    decision = model.decision_function(X)
    print(decision)

    learning_rate = model.learning_rate
    estimators = model.estimators_
    estimator_results = []
    for estimator in estimators:
        class_predictions = [tree.predict(X) for tree in estimator]
        estimator_results.append(np.array(class_predictions).T)

    final_predictions = np.sum(estimator_results, axis=0) * learning_rate
    print(final_predictions)

    return decision - final_predictions

# Test Datasets
explicar todas as instancias - tamanho medio da explicação & desvio padrão, porcentagem de redução de features:
 - para binario
 - para multiclasse
 - dataset de imagens
 - reordenar as features e reavaliar

## Iris Multiclass

In [89]:
gb_iris = GradientBoostingClassifier(n_estimators=100)

In [90]:
iris = load_iris()
X_iris, y_iris = iris.data, iris.target
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris, y_iris, test_size=0.1, random_state=101)

gb_iris.fit(X_iris_train, y_iris_train)

GradientBoostingClassifier()

In [91]:
# check_explanation = []
# for i in range(len(X_iris)):
#   if X_iris[i][3] == 1.6:
#     check_explanation.append((i, X_iris[i], y_iris[i], gb_iris.predict([X_iris[i]])))
# check_explanation

In [92]:
iris_explainer = Explainer(gb_iris, X_iris)

In [ ]:
iris_explain_sizes = pd.DataFrame(columns=['explain_size'])
iris_count_zeros = 0

for i in range(len(X_iris_test)):
    explain_size = len(iris_explainer.explain(X_iris_test[i], reorder='asc'))
    if explain_size == 0:
        iris_count_zeros += 1
    iris_explain_sizes.loc[len(iris_explain_sizes)] = [explain_size]

iris_explain_stat = pd.DataFrame({
    'mean': [iris_explain_sizes['explain_size'].mean()],
    'std': [iris_explain_sizes['explain_size'].std()],
    'count_zeros': [iris_count_zeros],
    'dataset_features': [iris.data.shape[1]],
})
iris_explain_stat

,mean,std,count_zeros,dataset_features
0,1.933333,0.258199,0,4


In [ ]:
iris_explain_sizes = pd.DataFrame(columns=['explain_size'])
iris_count_zeros = 0

for i in range(len(X_iris_test)):
    explain_size = len(iris_explainer.explain(X_iris_test[i], reorder=None))
    if explain_size == 0:
        iris_count_zeros += 1
    iris_explain_sizes.loc[len(iris_explain_sizes)] = [explain_size]

iris_explain_stat = pd.DataFrame({
    'mean': [iris_explain_sizes['explain_size'].mean()],
    'std': [iris_explain_sizes['explain_size'].std()],
    'count_zeros': [iris_count_zeros],
    'dataset_features': [iris.data.shape[1]],
})
iris_explain_stat

,mean,std,count_zeros,dataset_features
0,1.933333,0.258199,0,4


In [ ]:
iris_explain_sizes = pd.DataFrame(columns=['explain_size'])
iris_count_zeros = 0

for i in range(len(X_iris_test)):
    explain_size = len(iris_explainer.explain(X_iris_test[i], reorder='desc'))
    if explain_size == 0:
        iris_count_zeros += 1
    iris_explain_sizes.loc[len(iris_explain_sizes)] = [explain_size]

iris_explain_stat = pd.DataFrame({
    'mean': [iris_explain_sizes['explain_size'].mean()],
    'std': [iris_explain_sizes['explain_size'].std()],
    'count_zeros': [iris_count_zeros],
    'dataset_features': [iris.data.shape[1]],
})
iris_explain_stat

,mean,std,count_zeros,dataset_features
0,1.933333,0.258199,0,4


## Cancer Binary

In [96]:
gb_cancer = GradientBoostingClassifier(n_estimators=10)

In [97]:
cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target
X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(
    X_cancer, y_cancer, test_size=0.1, random_state=101)

gb_cancer.fit(X_cancer_train, y_cancer_train)

GradientBoostingClassifier(n_estimators=10)

In [98]:
cancer_explainer = Explainer(gb_cancer, X_cancer)

In [ ]:
cancer_explain_sizes = pd.DataFrame(columns=['explain_size'])
cancer_count_zeros = 0

for i in range(len(X_cancer_test)):
    explain_size = len(cancer_explainer.explain(
        X_cancer_test[i], reorder='asc'))
    if explain_size == 0:
        cancer_count_zeros += 1
        print(i)
    cancer_explain_sizes.loc[len(cancer_explain_sizes)] = [explain_size]

cancer_explain_stat = pd.DataFrame({
    'mean': [cancer_explain_sizes['explain_size'].mean()],
    'std': [cancer_explain_sizes['explain_size'].std()],
    'count_zeros': [cancer_count_zeros],
    'dataset_features': [cancer.data.shape[1]],
})
cancer_explain_stat

,mean,std,count_zeros,dataset_features
0,3.964912,1.224233,0,30


In [ ]:
cancer_explain_sizes = pd.DataFrame(columns=['explain_size'])
cancer_count_zeros = 0

for i in range(len(X_cancer_test)):
    explain_size = len(cancer_explainer.explain(
        X_cancer_test[i], reorder='desc'))
    if explain_size == 0:
        cancer_count_zeros += 1
        print(i)
    cancer_explain_sizes.loc[len(cancer_explain_sizes)] = [explain_size]

cancer_explain_stat = pd.DataFrame({
    'mean': [cancer_explain_sizes['explain_size'].mean()],
    'std': [cancer_explain_sizes['explain_size'].std()],
    'count_zeros': [cancer_count_zeros],
    'dataset_features': [cancer.data.shape[1]],
})
cancer_explain_stat

,mean,std,count_zeros,dataset_features
0,4.052632,1.394134,0,30


## Wine Multiclass

In [101]:
gb_wine = GradientBoostingClassifier(n_estimators=100)

In [102]:
wine = load_wine()
X_wine, y_wine = wine.data, wine.target
X_wine_train, X_wine_test, y_wine_train, y_wine_test = train_test_split(
    X_wine, y_wine, test_size=0.1, random_state=101)

gb_wine.fit(X_wine_train, y_wine_train)

GradientBoostingClassifier()

In [103]:
wine_explainer = Explainer(gb_wine, X_wine)

In [ ]:
wine_explain_sizes = pd.DataFrame(columns=['explain_size'])
wine_count_zeros = 0

for i in range(len(X_wine_test)):
    explain_size = len(wine_explainer.explain(X_wine_test[i], reorder='asc'))
    if explain_size == 0:
        wine_count_zeros += 1
        print(i)
    wine_explain_sizes.loc[len(wine_explain_sizes)] = [explain_size]

wine_explain_stat = pd.DataFrame({
    'mean': [wine_explain_sizes['explain_size'].mean()],
    'std': [wine_explain_sizes['explain_size'].std()],
    'count_zeros': [wine_count_zeros],
    'dataset_features': [wine.data.shape[1]],
})
wine_explain_stat

,mean,std,count_zeros,dataset_features
0,3.833333,0.785905,0,13


In [ ]:
wine_explain_sizes = pd.DataFrame(columns=['explain_size'])
wine_count_zeros = 0

for i in range(len(X_wine_test)):
    explain_size = len(wine_explainer.explain(X_wine_test[i], reorder='desc'))
    if explain_size == 0:
        wine_count_zeros += 1
        print(i)
    wine_explain_sizes.loc[len(wine_explain_sizes)] = [explain_size]

wine_explain_stat = pd.DataFrame({
    'mean': [wine_explain_sizes['explain_size'].mean()],
    'std': [wine_explain_sizes['explain_size'].std()],
    'count_zeros': [wine_count_zeros],
    'dataset_features': [wine.data.shape[1]],
})
wine_explain_stat

,mean,std,count_zeros,dataset_features
0,3.944444,0.725358,0,13


## Digits multiclass

In [106]:
gb_digits = GradientBoostingClassifier(n_estimators=5)

In [107]:
digits = load_digits()
X_digits, y_digits = digits.data, digits.target
X_digits_train, X_digits_test, y_digits_train, y_digits_test = train_test_split(
    X_digits, y_digits, test_size=0.01, random_state=101)

gb_digits.fit(X_digits_train, y_digits_train)

GradientBoostingClassifier(n_estimators=5)

In [108]:
digits_explainer = Explainer(gb_digits, X_digits)

In [ ]:
digits_explain_sizes = pd.DataFrame(columns=['explain_size'])
digits_count_zeros = 0

for i in range(len(X_digits_test)):
    explain_size = len(digits_explainer.explain(
        X_digits_test[i], reorder='asc'))
    if explain_size == 0:
        digits_count_zeros += 1
        print(i)
    digits_count = i
    digits_explain_sizes.loc[len(digits_explain_sizes)] = [explain_size]

digits_explain_stat = pd.DataFrame({
    'mean': [digits_explain_sizes['explain_size'].mean()],
    'std': [digits_explain_sizes['explain_size'].std()],
    'count_zeros': [digits_count_zeros],
    'dataset_len': [digits_count],
    'dataset_features': [digits.data.shape[1]],
})
digits_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,13.5,3.552133,0,17,64


In [ ]:
digits_explain_sizes = pd.DataFrame(columns=['explain_size'])
digits_count_zeros = 0

for i in range(len(X_digits_test)):
    explain_size = len(digits_explainer.explain(
        X_digits_test[i], reorder='desc'))
    if explain_size == 0:
        digits_count_zeros += 1
        print(i)
    digits_count = i
    digits_explain_sizes.loc[len(digits_explain_sizes)] = [explain_size]

digits_explain_stat = pd.DataFrame({
    'mean': [digits_explain_sizes['explain_size'].mean()],
    'std': [digits_explain_sizes['explain_size'].std()],
    'count_zeros': [digits_count_zeros],
    'dataset_len': [digits_count],
    'dataset_features': [digits.data.shape[1]],
})
digits_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,14.277778,3.785507,0,17,64


## Auto (classification)

In [195]:
gb_auto = GradientBoostingClassifier(n_estimators=100)

In [196]:
auto_data = fetch_data('auto')
auto_data.head()

,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,target
0,26,11,1,0,2,2,1,0,93.7,157.3,...,1,4,19,9.4,42,16,31.0,38.0,119,2
1,3,17,1,1,1,3,1,0,99.1,186.6,...,5,25,10,9.0,23,16,19.0,26.0,70,2
2,17,19,1,0,2,0,2,0,98.4,176.2,...,5,30,28,9.3,9,8,24.0,30.0,62,2
3,19,9,1,0,2,0,2,0,96.6,180.3,...,5,22,12,8.3,21,7,16.0,18.0,92,3
4,51,20,1,0,2,0,1,0,94.5,159.3,...,5,13,24,8.5,54,16,24.0,29.0,15,3


In [197]:
X_auto = auto_data.drop(columns=['target']).values
y_auto = auto_data['target'].values
X_auto_train, X_auto_test, y_auto_train, y_auto_test = train_test_split(
    X_auto, y_auto, test_size=0.1, random_state=101)

gb_auto.fit(X_auto_train, y_auto_train)

GradientBoostingClassifier()

In [198]:
auto_explainer = Explainer(gb_auto, X_auto)

In [199]:
auto_explain_sizes = pd.DataFrame(columns=['explain_size'])
auto_count_zeros = 0

for i in range(len(X_auto_test)):
    explain_size = len(auto_explainer.explain(X_auto_test[i], reorder='asc'))
    if explain_size == 0:
        auto_count_zeros += 1
        print(i)
    auto_count = i
    auto_explain_sizes.loc[len(auto_explain_sizes)] = [explain_size]

auto_explain_stat = pd.DataFrame({
    'mean': [auto_explain_sizes['explain_size'].mean()],
    'std': [auto_explain_sizes['explain_size'].std()],
    'count_zeros': [auto_count_zeros],
    'dataset_len': [auto_count],
    'dataset_features': [X_auto.shape[1]],
})
auto_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,25.0,0.0,0,20,25


In [200]:
auto_explain_sizes = pd.DataFrame(columns=['explain_size'])
auto_count_zeros = 0

for i in range(len(X_auto_test)):
    explain_size = len(auto_explainer.explain(X_auto_test[i], reorder='desc'))
    if explain_size == 0:
        auto_count_zeros += 1
        print(i)
    auto_count = i
    auto_explain_sizes.loc[len(auto_explain_sizes)] = [explain_size]

auto_explain_stat = pd.DataFrame({
    'mean': [auto_explain_sizes['explain_size'].mean()],
    'std': [auto_explain_sizes['explain_size'].std()],
    'count_zeros': [auto_count_zeros],
    'dataset_len': [auto_count],
    'dataset_features': [X_auto.shape[1]],
})
auto_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,25.0,0.0,0,20,25


## backache (classification)

In [204]:
gb_backache = GradientBoostingClassifier(n_estimators=100)

In [114]:
backache_data = fetch_data('backache')
backache_data

,id,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,...,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,target
0,1.0,1,0,26.0,1.52,54.5,75.0,3.35,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2.0,3,0,23.0,1.60,59.1,68.6,2.22,1,2,...,1,0,0,0,0,0,0,0,0,0
2,3.0,2,6,24.0,1.57,73.2,82.7,4.15,0,1,...,1,0,0,0,0,0,0,0,0,0
3,4.0,1,8,22.0,1.52,41.4,47.3,2.81,0,1,...,1,0,0,0,0,0,0,0,0,0
4,5.0,1,0,27.0,1.60,55.5,60.0,3.75,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,176.0,1,0,34.0,1.63,50.9,60.5,2.93,0,1,...,0,0,0,0,0,0,0,1,0,0
176,177.0,3,3,26.0,1.63,66.8,84.1,3.10,1,3,...,0,0,0,0,0,0,1,1,0,0
177,178.0,1,7,18.0,1.50,54.1,60.5,3.52,0,0,...,0,0,0,0,0,0,0,1,0,0
178,179.0,3,6,39.0,1.52,82.7,84.1,3.35,1,2,...,0,0,0,0,0,0,0,0,0,0


In [205]:
X_backache = backache_data.drop(columns=['target']).values
y_backache = backache_data['target'].values
X_backache_train, X_backache_test, y_backache_train, y_backache_test = train_test_split(
    X_backache, y_backache, test_size=0.1, random_state=101)

gb_backache.fit(X_backache_train, y_backache_train)

GradientBoostingClassifier()

In [206]:
backache_explainer = Explainer(gb_backache, X_backache)

In [207]:
backache_explain_sizes = pd.DataFrame(columns=['explain_size'])
backache_count_zeros = 0

for i in range(len(X_backache_test)):
    explain_size = len(backache_explainer.explain(X_backache_test[i], reorder='asc'))
    if explain_size == 0:
        backache_count_zeros += 1
        print(i)
    backache_count = i
    backache_explain_sizes.loc[len(backache_explain_sizes)] = [explain_size]

backache_explain_stat = pd.DataFrame({
    'mean': [backache_explain_sizes['explain_size'].mean()],
    'std': [backache_explain_sizes['explain_size'].std()],
    'count_zeros': [backache_count_zeros],
    'dataset_len': [backache_count],
    'dataset_features': [X_backache.shape[1]],
})
backache_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,8.277778,1.447332,0,17,32


In [208]:
backache_explain_sizes = pd.DataFrame(columns=['explain_size'])
backache_count_zeros = 0

for i in range(len(X_backache_test)):
    explain_size = len(backache_explainer.explain(X_backache_test[i], reorder='desc'))
    if explain_size == 0:
        backache_count_zeros += 1
        print(i)
    backache_count = i
    backache_explain_sizes.loc[len(backache_explain_sizes)] = [explain_size]

backache_explain_stat = pd.DataFrame({
    'mean': [backache_explain_sizes['explain_size'].mean()],
    'std': [backache_explain_sizes['explain_size'].std()],
    'count_zeros': [backache_count_zeros],
    'dataset_len': [backache_count],
    'dataset_features': [X_backache.shape[1]],
})
backache_explain_stat

,mean,std,count_zeros,dataset_len,dataset_features
0,10.111111,1.711171,0,17,32
